# 1. Preparation

### 1.1 Packages, Dates, & Initials

In [5]:
# plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#from labellines import labelLine, labelLines

#administrative packages
import os
import datetime
mydate = datetime.datetime.now()
from datetime import date,timedelta
import time
start = time.time()

#standard packages
import pandas as pd
import numpy as np
import math

# plotting
import matplotlib.pyplot as plt
%matplotlib inline
#from labellines import labelLine, labelLines

#administrative packages
import os
import operator
import datetime
mydate = datetime.datetime.now()
from datetime import date,timedelta
import time
start = time.time()
import logging
logger = logging.getLogger('ftpuploader')


holidays = [
    '2017-01-02',
    '2017-01-16',
    '2017-02-20',
    '2017-05-29',
    '2017-07-04',
    '2017-09-04',
    '2017-11-10',
    '2017-11-23',
    '2017-11-24',
    '2017-12-22',
    '2017-12-25',
    '2017-12-26',
    '2017-12-27',
    '2017-12-28',
    '2017-12-29',
    '2018-01-01',
    '2018-01-15',
    '2018-02-19',
    '2018-04-28',
    '2018-07-04',
    '2018-09-03',
    '2018-11-12',
    '2018-11-22',
    '2018-11-23',
    '2018-12-24',
    '2018-12-25',
    '2018-12-26',
    '2018-12-27',
    '2018-12-28',
    '2018-12-31',
    '2019-01-01',
    '2019-01-21',
    '2019-02-18',
    '2019-05-27',
    '2019-07-04',
    '2019-09-02',
    '2019-11-11',
    '2019-11-28',
    '2019-11-29',
    '2019-12-24',
    '2019-12-25',
    '2019-12-26',
    '2019-12-27',
    '2019-12-30',
    '2019-12-31',
    '2020-01-01',
    '2020-01-20',
    '2020-02-17',
    '2020-05-25',
    '2020-07-03',
    '2020-09-07',
    '2020-11-11',
    '2020-11-26',
    '2020-11-27',
    '2020-12-24',
    '2020-12-25',
    '2020-12-28',
    '2020-12-29',
    '2020-12-30',
    '2020-12-31',
    '2021-01-01'
]
holidays = set([pd.Timestamp(h) for h in holidays])


planner_initials = {
    'ABC': 'Alan Carreon',
    'AS': 'Ana Spinola',
    'AMS': 'Andrew Smith',
    'CG': 'Chip Griffin',
    'EB': 'Ethan T Bindernagel',
    'GDK': 'Greg Kapovich',
    'GV': 'Gerardo Victoria',
    'HEC': 'Haley E Croffoot',
    'HH': 'Haley Hubbard',
    'JG': 'Jessica J Gonzalez',
    'JCav': 'Jeanine Cavalli',
    'KN': 'Ken Nodder',
    'OA': 'Ozzy Arce',
    'SKG': 'Simar Gill',
    'SP': 'Sukhamrit S Purewal',
    'TC': 'Trishia Caguiat'
}
planner_names = {v:k for k,v in planner_initials.items()}
planner_names['Haley Hubbard'] = 'HEC'
planner_initials['HH'] = 'Haley E Croffoot'

#Complete_Closed = ['Approved - Closed', 'Closed', 'Not Approved - Closed']
#Planner_Closed = ['Close Out', 'Approved', 'Not Approved']
Ent = {
 'Zoning Amendment': 'Amend',
 'Use Permit Minor': 'MUP',
 'Design Review': 'DR',
 'Design Review Oversized Home': 'DR',
 'Design Review Other': 'DR',
 'Design Review Commercial': 'DR',
 'General Plan Amendment': 'Amend',
 'Design Review Antenna': 'DR',
 'Design Review Residential': 'DR',
 'Use Permit Conditional': 'CUP',
 'Use Permit Administrative': 'AUP',
 'ZCL':'ZCL',
 'Variance': 'Vari',
 'Rezoning': 'ReZone',
 'Tree Dripline Encroachment': 'Tree',
 'Tree Removal Permit': 'Tree',
 '': 'Other',
 'Other': 'Other',
 'Tentative Map Major Subdivision': 'Maj Sub',
 'Drip Line Encroachment': 'Other',
 'Tentative Map Minor Subdivision': 'Min Sub',
 'Tentative Map Condo Conversion': 'Conv',
 'Hillside Performance Standards': 'Other'}
Ent_names = {v:k for k,v in Ent.items()}
Ent_names['Other'] = 'Other'

TaskStatus = {
    'Building':{
        'Application Submittal - Route':'Start',
        'Consolidated Comments - With Customer for Response':'Round_End',
        'Resubmittal or Revision - Route':'Round_Start',
        #'Building Review - Notes':'Pause',
        'Ready to Issue - Conditionally Approved':'End',
        'Ready to Issue - Issued':'End',
        'Ready to Issue - Approved':'End'
    },
    'Site Development Permit':{
        'Status - Received':'Start',
        'Application Submittal - Route':'Start',
        'Consolidated Comments - With Customer for Response':'Round_End',
        'Consolidated Comments - Resubmittal':'Round_Start',
        #'Consolidated Comments - Ready to Issue': 'Round_Start',
        #'Status - Approved':'End'
        'Ready to Issue - Issue':'End'
        #'Application Submittal - Ready to Issue':'End'
    },
    'Planning':{
        'Status - Received':'Start',
        'Intake Review - Application Accepted':'Start',
        'Consolidated Comments - Deemed Incomplete':'Round_End',
        'Resubmittal - Route for Review':'Round_Start',
        'Consolidated Comments - Deemed Complete':'End',
        'Close Out - Approved':'End',
        'Staff Analysis - Set for Hearing':'End',
        'Staff Analysis - Staff Level Decision':'End',
        'Close Out - Not Approved - Closed':'End'
    }}

All_TaskStatus = []
[
    All_TaskStatus.extend(list(TSD.keys())) for dept,TSD in TaskStatus.items()
]
All_TaskStatus = set(All_TaskStatus)

sorts = ['Date Status', 'Updated Date']



### 1.2 Load Data

In [6]:
accela_directory = r'O:\CDD\PLANNING\AN\Projects\Accela Reporting\Combined'
dept_paths = {
    'Planning':'PermitMetrics_Planning.csv',
    'Building':'PermitMetrics_Building.csv',
    'Site Development Permit':'PermitMetrics_SDP.csv'
          }
def combine_dept_permits(path_dictionary, directory_path):
    os.chdir(directory_path)
    
    PLAN = pd.read_csv(path_dictionary['Planning'])
    lu = ['Residential', 'Commercial']
    PLAN['Permit Type'].fillna(value='', inplace=True)
    PLAN['Type of Work'] = PLAN['Permit Type'].apply(lambda s: [s for s in s.split(' ') if s in lu][0] if any(l in s for l in lu) else '')
    PLAN['Record Type'] = 'Planning'
    
    SDP = pd.read_csv(path_dictionary['Site Development Permit'])
    SDP['Type of Work'] = SDP['Permit Type'].apply(lambda s: 'Commercial' if s == 'COMM' else s)
    SDP.loc[SDP['Type of Work']=='Subdivision', 'Type of Work'] = 'Residential'
    SDP['Type of Work'].unique()

    BUILD = pd.read_csv(path_dictionary['Building'])
    BUILD.drop(columns='Construction Type', inplace=True)  #['Permit Type'] = BUILD['Construction Type'].apply(lambda x: const_dict[x]['subtype'] if x>0 else '')
    const_dict = {
     101: {'type of work': 'Residential',
      'subtype': 'Construction',
      'definition': 'Single Family Houses - Detached'},
     102: {'type of work': 'Residential',
      'subtype': 'Construction',
      'definition': 'Single Family Houses - Attached'},
     103: {'type of work': 'Residential',
      'subtype': 'Construction',
      'definition': 'Two Family Buildings'},
     104: {'type of work': 'Residential',
      'subtype': 'Construction',
      'definition': 'Three and Four Family Buildings'},
     105: {'type of work': 'Residential',
      'subtype': 'Construction',
      'definition': 'Five or More Family Buildings'},
     106: {'type of work': 'Residential',
      'subtype': 'Construction',
      'definition': 'ADU (New Detached, Additions & Conversrion)'},
     109: {'type of work': 'Residential',
      'subtype': 'Construction',
      'definition': 'Other (Including Residential Foundation only permit)  '},
     213: {'type of work': 'Commercial',
      'subtype': 'Construction',
      'definition': 'Hotels, Motels, and Tourist Cabins'},
     214: {'type of work': 'Commercial',
      'subtype': 'Construction',
      'definition': 'Other No Housekeeping Shelter'},
     318: {'type of work': 'Commercial',
      'subtype': 'Construction',
      'definition': 'Amusement, Social, and Recreational'},
     319: {'type of work': 'Commercial',
      'subtype': 'Construction',
      'definition': 'Churches and Other Religeous Buildings'},
     320: {'type of work': 'Commercial',
      'subtype': 'Construction',
      'definition': 'Industrial Buildings'},
     321: {'type of work': 'Commercial',
      'subtype': 'Construction',
      'definition': 'Parking Garages (Buildings & Open Decked)'},
     322: {'type of work': 'Commercial',
      'subtype': 'Construction',
      'definition': 'Service Stations and Repair Garages'},
     323: {'type of work': 'Commercial',
      'subtype': 'Construction',
      'definition': 'Hospitals and Institutional Buildings'},
     324: {'type of work': 'Commercial',
      'subtype': 'Construction',
      'definition': 'Office, Bank, and Professional Buildings'},
     325: {'type of work': 'Commercial',
      'subtype': 'Construction',
      'definition': 'Public Works and Utilities Buildings'},
     326: {'type of work': 'Commercial',
      'subtype': '',
      'definition': 'Schools and Other Educational Buildings'},
     327: {'type of work': 'Commercial',
      'subtype': 'Construction',
      'definition': 'Stores and Customer Service'},
     328: {'type of work': 'Commercial',
      'subtype': 'Construction',
      'definition': 'Other Nonresidential Buildings (Including foundation only permit)'},
     329: {'type of work': 'Commercial',
      'subtype': 'Construction',
      'definition': 'Structures Other Than Buildings'},
     434: {'type of work': 'Commercial',
      'subtype': 'Additions',
      'definition': 'Additions, Alterations and Conversions - Residential (Includes A.D.U.)'},
     437: {'type of work': 'Commercial',
      'subtype': 'Additions',
      'definition': 'Additions, Alterations and Conversions - Nonresidential and No Housekeeping'},
     438: {'type of work': 'Residential',
      'subtype': 'Additions',
      'definition': 'Additions of Residential Garages and Carports'},
     645: {'type of work': 'Residential',
      'subtype': 'DEMO',
      'definition': 'DEMO Single Family Houses'},
     646: {'type of work': 'Residential',
      'subtype': 'DEMO',
      'definition': 'DEMO Two Family Buildings'},
     647: {'type of work': 'Residential',
      'subtype': 'DEMO',
      'definition': 'DEMO Three and Four Family Buildings'},
     648: {'type of work': 'Residential',
      'subtype': 'DEMO',
      'definition': 'DEMO Five or More Family Buildings'},
     649: {'type of work': 'Commercial',
      'subtype': 'DEMO',
      'definition': 'DEMO All Other Buildings and Structures'}
    }
    
    p_cols = [c for c in list(PLAN) if c not in set(list(BUILD)+list(SDP))]
    p_not_cols = [c for c in set(list(BUILD)+list(SDP)) if c not in list(PLAN)]
    if len(p_cols+p_not_cols) > 0:
        print("Planning cols that are not in the rest: ", p_cols)
        print("Planning cols that are missing from the rest: ", p_not_cols)
    
    b_cols = [c for c in list(BUILD) if c not in set(list(PLAN)+list(SDP))]
    b_not_cols = [c for c in set(list(PLAN)+list(SDP)) if c not in list(BUILD)]
    if len(b_cols+b_not_cols) > 0:
        print("Building cols that are not in the rest: ", b_cols)
        print("Building cols that are missing from the rest: ", b_not_cols)
        
    s_cols = [c for c in list(SDP) if c not in set(list(BUILD)+list(PLAN))]
    s_not_cols = [c for c in set(list(BUILD)+list(PLAN)) if c not in list(SDP)]
    if len(s_cols+s_not_cols) > 0:
        print("SDP cols that are not in the rest: ", s_cols)
        print("SDP cols that are missing from the rest: ", s_not_cols)

    All_Depts = pd.concat([PLAN, SDP, BUILD])
    Extra_Types = [d for d in All_Depts['Record Type'].unique() if d not in dept_paths.keys()]
    if len(Extra_Types)>0:
        print(Extra_Types)
    
    All_Depts['Address'].fillna(value='')
    All_Depts['Address'] = All_Depts['Address'].apply(
        lambda a: ' '.join([
            aa.capitalize() for aa in a.split(', ')[0].split(' ')
        ]) if type(a)==str else str(a)
    )
    
    All_Depts.loc[All_Depts['Type of Work']=='RES', 'Type of Work'] = 'Residential'
    
    return All_Depts

Permits = combine_dept_permits(dept_paths, accela_directory)

P = Permits.copy()

# 2. Cleaning

### 2.1 Format Table

In [7]:
Permits = P.copy()

Record_cols = [
        'Permit #',
        #'Task Status',
        'Round Status',
        'Date Assigned',
        'Date Status',
        'Updated Date'
              ]
Admin_cols = ['Permit #',
             'Record Type',
             'Type of Work',
             'Permit Type',
             'Record Status',
             'Open Date',
             'Description',
             'APN',
             'Address'
             ]

def format_data(P_df):
    
    # turns date columns into datetime format
    date_cols = [col for col in list(P_df) if ('Date ' in col)or(' Date' in col)]
    P_df[date_cols] = P_df[date_cols].apply(pd.to_datetime)
    
    # Column identifying the record's workflow position through the completed task
    P_df['Task Status'] = P_df['Task'] + ' - ' + P_df['Status']
    
    # Column identifying the record's round position
    # non-NaN records are tasks that end or start a round through accepting or denying a submitted application
    P_df['Round Status'] = P_df['Task Status']
    
    # By looping by department/permit type, the script can apply nested, translated dictionaries 
    # The TaskStatus dictionary translates the task statuses into the round statuses
    # If the TaskStatus isn't in the department's nested dictionary, it receives an NaN
    for dept in P_df['Record Type'].unique():
        P_df.loc[P_df['Record Type']==dept, 'Round Status'] = P_df.loc[P_df['Record Type']==dept, 'Round Status'].map(TaskStatus[dept])
    
    # This filters out the NaN records that did not give a round status
    #P_df = P_df.loc[~P_df['Round Status'].isna()].copy()
    
    # Removes the duplicates
    P_df.drop_duplicates(subset=['Task Status']+Record_cols, inplace=True)
    
    return P_df

Permits = format_data(Permits)

def Planning_Formats(Plan_df):
    raw_records = Plan_df.set_index('Permit #')

    # For Planning Applications, allow received date
    if 'Status - Received' in temp.index.get_level_values(1):

        temp.reset_index(inplace=True)
        focus = set(temp[temp['Task Status'].isin(['Status - Assigned', 'Status - Received'])].index)
        #IDs = {i:[] for i in focus}
        #[IDs[i].append(ts) for i in focus]
        #IDs = [k for k,v in IDs.items() if len(v) > 1]
        for x in focus:
            temp.loc[x, 'DATE ASSIGNED'] = temp.loc[x, 'DATE STATUS']
            temp.loc[x, 'DATE STATUS'] = temp.loc[x, 'UPDATED DATE']
        temp.set_index(['Permit #', 'Task Status'], inplace=True)
    
    Plan_Review = raw_records.loc[(raw_records['INTAKER'] != ' '), 'INTAKER'].map(planner_names)

    Assigned = raw_records[raw_records['Task Status']=='Planning Review - Assigned - Notify Applicant'].copy()
    Assigned['INTAKER'] = Assigned['INTAKER'].map(planner_names)
    Assigned = Assigned['INTAKER'].to_dict()

    # Determine project planner
    ## Typically most common planner on project
    def find_planner(series):
        planner_counts = series.value_counts().sort_values(ascending=True).to_dict()

        planner_list = [plnr for plnr, v in sorted(planner_counts.items(), key=lambda item: item[1], reverse=True) 
                        if (plnr in planner_initials.keys()) and (plnr not in ['GV', 'AS']) and (type(plnr)==str)]
        if len(planner_list) == 0:
            update_planner = 'GV'
        else:
            update_planner = planner_list[0]
        try:
            return update_planner
        except:
            print(planner_counts, planner_list)
            return ''

    Plan_prep = raw_records['INTAKER'].map(planner_names)

    raw_records = raw_records.drop(columns='INTAKER')

    Plan_prep = pd.concat((Plan_prep, Plan_Review))
    Plan_prep.name = 'INTAKER'

    Plan_prep = pd.DataFrame(Plan_prep).reset_index()

    Plan_prep = Plan_prep[~Plan_prep['INTAKER'].isna()]

    Planners = Plan_prep.groupby('Permit #').agg(INTAKER=('INTAKER',find_planner))

    raw_records = raw_records.join(Planners)

    raw_records = raw_records.reset_index().drop_duplicates()

    raw_records.loc[raw_records['Permit #'].isin(Assigned.keys()),'INTAKER'] = raw_records.loc[raw_records['Permit #'].isin(Assigned.keys()),'Permit #'].map(Assigned)

    def straight_record(name):
        name = str(name).title().strip()
        name = name.replace("'S", "'s").replace("Sfr", "SFR").replace(" Wc ", " WC ").replace("Zcl","ZCL").replace("Aup","AUP").replace("Cup","CUP")
        if len(name) > 30:
            for a in ['@', ' At ', '&', ' And ']:
                if a in name:
                    name = name.split(a)[0].strip()
        if len(name) > 40:
            for a in ['&', ' And ']:
                if a in name:
                    name = name.split(a)[0].strip()
        if len(name) > 45:
            name = ' '.join(name.split(' ')[:7])
        return name

    raw_records['Name'] = raw_records['RECORD NAME'].apply(lambda a: straight_record(a))

    raw_fields.extend(['Name','INTAKER'])
    
    return raw_fields

pts = [' '.join([i,ts,str(d),str(dd),str(ddd)]) for i,ts,d,dd,ddd in Permits[['Permit #', 'Task Status', 'Date Assigned', 'Date Status', 'Updated Date']].values]

print(len(set(pts)))
print(len(pts))

Admin = Permits[Admin_cols].copy().drop_duplicates().set_index('Permit #')

Records = Permits[Record_cols].sort_values(['Permit #']+sorts).set_index(['Permit #', 'Round Status'])

Records['Index'] = 0
def indexer(R_df):
    R_df.sort_values(by=sorts, inplace=True)
    R_df.reset_index(inplace=True, drop=False)
    R_df['Index'] = R_df.index
    R_df.set_index('Round Status', inplace=True)
    
    return R_df['Index']

Records['Index'] = Records.groupby(level=[0]).apply(indexer)

Records = Records[['Index', 'Date Assigned', 'Date Status']]

dept_dict = Permits[['Record Type','Permit #']].drop_duplicates().set_index('Permit #')['Record Type'].to_dict()

r = Records.copy()

27771
27771


In [8]:
Records = r.copy()

sum_cols = [
        'Total Days', 
        'Working Days', 
        'Rounds', 
        'Days till Routing', 
        'Days per Round', 
        'Submittal Date', 
        'Complete Date'
    ]

def sum_days(strt, nd, busi=True):
    if busi == True:
        days = [d for d in pd.bdate_range(start=strt, end=nd).to_list() if d not in holidays]
    if busi == False:
        days = pd.date_range(start=strt, end=nd).to_list()
    return days

def check_days(srt, ned, a_list, t_list):
    s = [ss for ss in sum_days(srt, ned, busi=True) if ss not in t_list]
    t_list.update(s)
    a_list.append(len(s))
    return a_list, t_list

def round_check(test_df, test, dept):
    index = 'Index'
    
    
    
    if len(np.where(test[:,0] == 'End')[0]) > 1:
        while len(np.where(test[:,0] == 'End')[0]) > 1:
            false_end = np.where(test[:,0] == 'End')[0][-1]
            test = np.delete(test, false_end, 0)
    elif len(np.where(test[:,0] == 'End')[0]) == 0:
        test[-1,0] = 'End'

    if (len(test) == 1) and (len(test) != len(test_df)): 
        if test[0,1] > 0:
            start_date = pd.to_datetime(test_df.loc[test_df[index] == 0, 'Date Assigned'].values[0])
            route_date = pd.to_datetime(test_df.loc[test_df[index] == 0, 'Date Status'].values[0])
            test = np.vstack([np.array(['Start', 0, start_date, route_date]), test])
            n = True
    else:
        n = False
    round_check = {
        'Start':1,
        'Round_Start':1,
        'Round_End':0,
        'End':0
    }
    
    end_gap = False
    if (dept != 'Building'):
        if(test[-1,0] == 'End')&(len(test[:,0])>1):
            if(test[-2,0] == 'Round_End'):
                end_gap = True
                
    check = [round_check[l] for l in list(test[:,0])]

    f = 0
    for e,c in enumerate(check):
        if e == 0:
            if (c == 0)&(len(check)>1):
                test = np.insert(test, e + f, np.array(('temp_start', 
                                                            test[e+f,1], 
                                                            test[e+f,2], 
                                                            test[e+f,2])), 
                                     0)  
                f += 1
            pass
        else:
            if c != cc:
                pass
            elif c == cc:
                if c == 0:
                    test = np.insert(test, e + f, np.array(('temp_start', 
                                                            test[e+f,1], 
                                                            test[e+f,2], 
                                                            test[e+f,2])), 
                                     0)  
                    f += 1
                elif (c == 1)and(n==False):
                    #try:
                    pindex = test[e+f,1]
                    #except:
                    #    print(false_index)
                    if (pindex > 2)&(pindex-2 != test[e+f-1,1]): 
                        false_index = pindex-2
                    else:
                        false_index = pindex-1
                    try:
                        false_date = pd.to_datetime(test_df.loc[test_df[index] == false_index, 'Date Status'].values[0])
                    except:
                        print(index, test)
                        print(test_df.loc[test_df[index] == index, 'Date Status'])
                        break
                    test = np.insert(test, e + f, np.array(('temp_end', 
                                                                false_index, 
                                                                false_date, 
                                                                false_date)), 
                                         0)
                    #except Exception as exc:
                    #    print(false_index, index, e)
                    #    logger.error(str(exc))

                    f += 1
        ee = e
        cc = c
    if (ee > 0) & (cc == 1):
        pindex = test[-1,1]
        false_index = pindex
        mx = test_df[index].max()-1
        if mx == pindex:
            mx = test_df[index].max()
        false_date = pd.to_datetime(test_df.loc[test_df[index] == mx, 'Date Status'].values[0])
        test = np.insert(test, len(test), np.array(('temp_end', 
                                                false_index, 
                                                false_date, 
                                                false_date)), 
                         0)  
    return test[:,2:], end_gap

def find_round_length(array, startindex):

    if len(array) > 1:
        endindex = startindex + 1
    else:
        endindex = startindex

    startdate = array[startindex]
    enddate = array[endindex]
    round_length = len(sum_days(startdate, enddate, busi=True))
    return round_length

def describe_rounds(rounds_arr, double_end_problem=False):
    SUBMITTAL_DATE = rounds_arr[0][0]
    ROUTING_DATE = rounds_arr[0][1]
    if (pd.isna(SUBMITTAL_DATE))or(pd.isnull(SUBMITTAL_DATE)):
        SUBMITTAL_DATE = ROUTING_DATE
        print('first blank')
    if (SUBMITTAL_DATE > ROUTING_DATE):
        SUBMITTAL_DATE = ROUTING_DATE 
    if (pd.isna(SUBMITTAL_DATE))or(pd.isnull(SUBMITTAL_DATE)):
        SUBMITTAL_DATE = rounds_arr[1][1]
        print('second blank')
    END_DATE = rounds_arr[-1][1]
        
    
    DAYS_TILL_ROUTING = len(sum_days(SUBMITTAL_DATE, ROUTING_DATE, busi=True))-1
    TOTAL_DAYS = len(sum_days(SUBMITTAL_DATE, END_DATE, busi=False))

    updates = rounds_arr[:,1]

    if ((len(updates) % 2) != 0)and(len(updates)>1):
        print('not even', len(updates))
        
    DAYS_PER_ROUND = [find_round_length(updates, i) for i in range(0,len(updates), 2)]

    if double_end_problem == True:
        del DAYS_PER_ROUND[-1]
    
    ROUNDS_COUNT = len(DAYS_PER_ROUND)

    PROCESSING_DAYS = sum(DAYS_PER_ROUND)
    
    return [TOTAL_DAYS, PROCESSING_DAYS, ROUNDS_COUNT, DAYS_TILL_ROUTING, DAYS_PER_ROUND, SUBMITTAL_DATE, END_DATE], rounds_arr[:,1]

def RoundLengths(R_df):
    
    R_df = R_df.reset_index()
    
    record_id = R_df['Permit #'].values[0]
    dept = dept_dict[record_id]
    
    R_array = R_df[~R_df['Round Status'].isna()][['Round Status', 'Index', 'Date Assigned', 'Date Status']].copy().values
    if len(R_array)==0:
        R_array = R_df.loc[R_df['Index']==0, ['Round Status', 'Index', 'Date Assigned', 'Date Status']].copy().values
    
    RoundLength_array, end_check = round_check(R_df, R_array, dept)
    
    double_end_problem = False
    if (end_check == True)and(dept == 'Site Development Permit'):
        double_end_problem = True

    described_list, rounds = describe_rounds(RoundLength_array, double_end_problem=double_end_problem)

    new_row = described_list
    
    return new_row


Sums = Records.groupby(level=[0]).apply(RoundLengths).apply(lambda x:pd.Series(x))
Sums.columns = sum_cols



first blank
first blank
first blank
first blank
first blank
first blank
first blank
first blank
first blank
first blank
first blank
first blank
first blank
first blank


In [9]:
Final_Permits = Admin.join(Sums)
for d in [col for col in list(Final_Permits) if 'Date' in col]:
    Final_Permits[d.replace('Date','Quarter')] = Final_Permits[d].dt.to_period("Q")
    Final_Permits[d.replace('Date','Month')] = Final_Permits[d].dt.to_period('M')
    
Final_Permits['Rounds & OTC'] = Final_Permits['Rounds'].copy()
Final_Permits['Over-the-Counter'] = False

Final_Permits.loc[(Final_Permits['Rounds']>1), 'Rounds & OTC'] = Final_Permits.loc[(Final_Permits['Rounds']>1), 'Rounds'].apply(lambda s: str(s) + ' Rounds')
Final_Permits.loc[(Final_Permits['Rounds']>4), 'Rounds & OTC'] = '5+ Rounds'
Final_Permits.loc[(Final_Permits['Rounds']<=1), 'Rounds & OTC'] = '1 Round'
Final_Permits.loc[(Final_Permits['Total Days']<=1), 'Rounds & OTC'] = 'Over-the-Counter'

Final_Permits.loc[(Final_Permits['Total Days']<=1), 'Over-the-Counter'] = True

Final_Permits[Final_Permits['Rounds']==1]

,Record Type,Type of Work,Permit Type,Record Status,Open Date,Description,APN,Address,Total Days,Working Days,...,Submittal Date,Complete Date,Open Quarter,Open Month,Submittal Quarter,Submittal Month,Complete Quarter,Complete Month,Rounds & OTC,Over-the-Counter
Permit #,,,,,,,,,,,,,,,,,,,,,
B180001,Building,Residential,Alteration,Finaled,2018-01-02,"REPLACE VALUE, WETWALL, TILE AT SHOWER WALLS",180230027,3175 Walnut Blvd,1,1,...,2018-01-02,2018-01-02,2018Q1,2018-01,2018Q1,2018-01,2018Q1,2018-01,Over-the-Counter,True
B180005,Building,Residential,Alteration,Finaled,2018-01-02,REPLACE 3 WINDOWS AND 1 SLIDING PATIO DOOR. LI...,179280135,1251 Homestead Ave,1,1,...,2018-01-02,2018-01-02,2018Q1,2018-01,2018Q1,2018-01,2018Q1,2018-01,Over-the-Counter,True
B180006,Building,Commercial,Sign,Finaled,2018-01-02,"EXTERIOR SIGN FOR ""VAYA""",174220051,1800 Lacassie Av,1,1,...,2018-01-02,2018-01-02,2018Q1,2018-01,2018Q1,2018-01,2018Q1,2018-01,Over-the-Counter,True
B180007,Building,Residential,Alteration,Finaled,2018-01-02,KITCHEN REMODEL,182202016,1092 Hillendale Ct,1,1,...,2018-01-02,2018-01-02,2018Q1,2018-01,2018Q1,2018-01,2018Q1,2018-01,Over-the-Counter,True
B180008,Building,Residential,Alteration,Finaled,2018-01-02,KITCHEN REMODEL,171062065,179 Pioneer Ave,1,1,...,2018-01-02,2018-01-02,2018Q1,2018-01,2018Q1,2018-01,2018Q1,2018-01,Over-the-Counter,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Y20-064,Planning,,Other,Approved - Closed,2020-09-25,355 N WIGET LANE - ZCL,143140002,355 N Wiget Ln,18,12,...,2020-09-25,2020-10-12,2020Q3,2020-09,2020Q3,2020-09,2020Q4,2020-10,1 Round,False
Y20-071,Planning,,Other,Approved - Closed,2020-10-08,T-MOBILE GENERATOR @ 2968 DEERPARK DR - ZCL,142261012,2968 Deerpark Dr,182,120,...,2020-10-08,2021-04-07,2020Q4,2020-10,2020Q4,2020-10,2021Q2,2021-04,1 Round,False
Y20-077,Planning,Commercial,Design Review Commercial,Close Out,2020-10-30,HICKORY PIT PERGOLA,184220014,1495 S Main St,26,17,...,2020-10-30,2020-11-24,2020Q4,2020-10,2020Q4,2020-10,2020Q4,2020-11,1 Round,False


### 5.2 Export Excel Sheets

In [11]:
import datetime

def colnum_string(n):
    string = ""
    n = n + 1
    while n > 0:
        n, remainder = divmod(n - 1, 26)
        string = chr(65 + remainder) + string
    return string

def rng_create(A, one, B, two):
    return str(colnum_string(A)) + str(one) + ":" + str(colnum_string(B)) + str(two)

def get_col_widths(dataframe):
    # First we find the maximum length of the index column   
    idx_max = max([len(str(s)) for s in dataframe.index.values] + [len(str(dataframe.index.name))])
    # Then, we concatenate this to the max of the lengths of column name and its values for each column, left to right
    return [idx_max] + [max([len(str(s)) for s in dataframe[col].values] + [len(col)]) for col in dataframe.columns]

def normalize_types(lyr):
        str_list = list(lyr.select_dtypes(include=['datetime','period[Q-DEC]','O']))
        convert_dict = {s:str for s in str_list}
        no_str = [l for l in list(lyr) if l not in str_list]
        no_dict = lyr[no_str].dtypes.to_dict()
        convert_dict.update(no_dict)
        return lyr.astype(convert_dict)

new_path = 'CDD_Permits.xlsx'

import xlsxwriter as xlsx
wrkbk = xlsx.Workbook(new_path)
for OG, name in [(Final_Permits.fillna(value=''),'CDD_Permits')
                ]:
    worksheet = wrkbk.add_worksheet(name)
    
    lyr = OG.copy()
    
    lyr = normalize_types(lyr)
    
    lyr = lyr.fillna(value='')
    
    header = np.array([[lyr.index.name] + list(lyr)])
    data = lyr.reset_index().to_numpy()
    if data.shape[1] != header.shape[1]:        
        dif = data.shape[1] - header.shape[1]
        fix_head = [lyr.index.name] + list(lyr)
        for r in range(dif):
            fix_head = [' '] + fix_head
        header = np.array([fix_head])
    table = np.vstack((header,data))
    numRows,numColumns = table.shape
    
    idx = [0]
    if type(lyr.index) == pd.MultiIndex:
        idx.append(1)
        table[0,0:2] = list(lyr.index.names)
    else:
        table[0,0:1] = list(lyr.index.names)

    for c in range(numColumns):
        try:
            for r in range(numRows):
                format_dict = {'font_name':'Arial'}

                # TOP
                if (r == 0) & (c not in idx):
                    format_dict['bottom'] = 2
                    format_dict['bold'] = True
                    format_dict['text_wrap'] = True
                # INDEX NAME
                if (r == 0) & (c in idx):
                    format_dict['italic'] = True
                # RIGHT
                elif (r != 0) & (c == range(numColumns)[-1]):
                    format_dict['right'] = 2
                    if (r == range(numRows)[-1]):
                        format_dict['bottom'] = 2
                # LEFT
                elif (r != 0) & (c in idx):
                    format_dict['bold'] = True
                    if (1 not in idx)or(c == 1):
                        format_dict['right'] = 2
                # BOTTOM
                elif (r == range(numRows)[-1]) & (c not in idx):
                    format_dict['bottom'] = 2
                    if (c == range(numColumns)[-1]):
                        format_dict['right'] = 2
                item = table[r][c]
                if item == np.nan:
                    item = ''

                cell_format = wrkbk.add_format(format_dict)
                worksheet.write(r, c, item, cell_format)
            if c not in idx:
                worksheet.set_column(c, c, 10)
            else:
                worksheet.set_column(c, c, len(max(table[:,c], key=len))+1)
        except:
            print(table[r,c])
            
    worksheet.add_table('{}:{}'.format(colnum_string(0)+str(1),colnum_string(numColumns-1)+str(numRows)),
                       {'columns':[{'header':c} for c in header[0]]})
            
wrkbk.close()
